# Test AutoFileGroup
This notebook tests the grouping functionality in AutoFileGroup class

In [1]:
import os
import sys
import yaml
from loguru import logger

from autocoder.agent.auto_filegroup import AutoFileGroup
import byzerllm

## Create test data directory and files

In [3]:
# Create test directory
test_dir = ".tmp/test_auto_filegroup"
os.makedirs(test_dir, exist_ok=True)

# Create test YAML files
test_data = [
    {
        "name": "001_feature1.yml",
        "content": {
            "query": "Add input validation to login function",
            "urls": [
                "/src/auth/login.py",
                "/src/utils/validation.py"
            ]
        }
    },
    {
        "name": "002_feature2.yml",
        "content": {
            "query": "Implement password reset functionality",
            "urls": [
                "/src/auth/password_reset.py",
                "/src/auth/email.py"
            ]
        }
    },
    {
        "name": "003_feature3.yml", 
        "content": {
            "query": "Add caching to database queries",
            "urls": [
                "/src/db/cache.py",
                "/src/db/queries.py"
            ]
        }
    },
    {
        "name": "004_feature4.yml",
        "content": {
            "query": "Implement rate limiting for API endpoints",
            "urls": [
                "/src/api/rate_limiter.py",
                "/src/api/middleware.py"
            ]
        }
    },
    {
        "name": "005_feature5.yml",
        "content": {
            "query": "Add email verification after signup",
            "urls": [
                "/src/auth/email.py",
                "/src/auth/signup.py"
            ]
        }
    }
]

# Write test files
for item in test_data:
    file_path = os.path.join(test_dir,"actions", item["name"])
    with open(file_path, "w", encoding="utf-8") as f:
        yaml.dump(item["content"], f)

print(f"Created {len(test_data)} test files in {test_dir}")

Created 5 test files in .tmp/test_auto_filegroup


## Initialize LLM and test grouping

In [1]:
import os
import sys
import yaml
from loguru import logger

from autocoder.agent.auto_filegroup import AutoFileGroup
import byzerllm

llm = byzerllm.ByzerLLM.from_default_model(model="doubao_128k_chat")

# Create AutoFileGroup instance
grouper = AutoFileGroup(
    llm=llm, 
    project_dir="/Users/allwefantasy/projects/auto-coder", 
    file_size_limit=100)

# Test file grouping
groups = grouper.group_files()

print("\nFile Grouping Results:")
print("=====================")
for idx, group in enumerate(groups, 1):
    print(f"\nGroup {idx}: {group.name}")
    print("Queries:")
    for query in group.queries:
        print(f"- {query}")
    print("Files:")
    for url in group.urls:
        print(f"- {url}")

2024-12-21 12:11:49.130 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-12-21 12:11:49,212	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-12-21 12:11:49,244	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 



File Grouping Results:

Group 1: AutoFileGroup 功能优化与修复
Queries:
- 修复 @@AutoFileGroup(location: src/autocoder/agent/auto_filegroup.py) 中 action_files 排序逻辑错误
- 优化下 @@group_by_similarity(location: src/autocoder/agent/auto_filegroup.py)
- 在 notebooks/tests/test_auto_filegroup.ipynb 中, 修改导入路径和输出格式, 并添加执行结果输出. 在 src/autocoder/agent/auto_filegroup.py 中, 添加 FileGroup 和 FileGroups 模型, 并修改 group_by_similarity 方法返回类型为 FileGroups.
- 在 notebooks/tests 里新建一个jupyter notebook ,测试 @src/autocoder/agent/auto_filegroup.py 中的分组功能
- 修改 @@AutoFileGroup(location: src/autocoder/agent/auto_filegroup.py) 的初始化参数和 group_by_similarity 方法，增加 file_size_limit 限制和调整输入格式
- 重构 @src/autocoder/agent/auto_filegroup.py 的代码,我们分组逻辑是,给定一组yaml文件,得到里面的query和urls, 一起给到一个分组 prompt 函数,该函数prompt 函数会输出如下格式 分组名: 对应的子query列表, 对应的 urls 列表,以 json格式输出
- 在 src/autocoder/agent 目录下新增一个 auto_filegroup.py 文件, 使用prompt 函数实现一个自动分组功能. 参考 @src/autocoder/utils/__init__.py 文件中遍历 actions_dir 得到所有 yaml file, yaml file 内容典型如下:auto_merge: editblock
File

## Cleanup

In [6]:
# Remove test directory and files
import shutil
shutil.rmtree(test_dir)
print(f"\nCleaned up test directory: {test_dir}")


Cleaned up test directory: .tmp/test_auto_filegroup
